# シェルソート


挿入ソートでは、未整理のデータから取り出した値を、整理済みデータのどこに挿入すべきかがパッと分かると効率が良い。

そこで、なるべく整理済みのデータの最後尾（または先頭）に位置づけられるように前処理をしてしまおう、というのがシェルソートである。

（逆に、整理済みデータから挿入箇所を探すところを工夫したのがバイナリソート（二分探索挿入ソート）となる。）


シェルソートでは、未整理の配列$A$を間隔 g ごとの配列 $A_{modg0} = [a_0, a_g, a_2g, ...], A_{modg1} = [a_1, a_{g+1}, ...], ...$ に分けたうえで挿入ソートを行い、ラフに前処理をする。

この挿入ソートを行う際に、部分配列の最後尾に注目するやり方（螺旋本ではこちら）の他に、g 種類の部分配列ごとに挿入ソートを終わらせるやり方があると思われる。本実装は後者で行う。


In [1]:
from logging import getLogger, StreamHandler, DEBUG

logger = getLogger(__name__)
handler = StreamHandler()
handler.setLevel(DEBUG)
logger.setLevel(DEBUG)
logger.addHandler(handler)
logger.propagate = False

In [9]:
def insertion_sort(nums: list[int], g: int):
    # mod(g)==0, mod(g)==1...の部分配列ごとに挿入ソートする。
    for rem in range(0, g):
        logger.debug(f"{g=}, {rem=}")
        # 1周目は [rem, rem+g] を、2週目は [rem, rem+g, rem+2g] を...のように挿入ソートを行う。
        last = rem + g
        while last < len(nums):
            logger.debug(f"{g=}, {rem=}, {last=}")
            # challenger を g づつ減らしてループする。
            challenged = last - g
            while 0 <= challenged:
                logger.debug(f"{g=}, {rem=}, {last=}, {challenged=}")
                if nums[challenged] > nums[challenged + g]:
                    tmp = nums[challenged]
                    nums[challenged] = nums[challenged + g]
                    nums[challenged + g] = tmp
                challenged -= g
                logger.debug(f"{nums=}")

            last += g
            logger.debug(f"{nums=}")

        logger.debug(f"{nums=}")

    return nums

In [10]:
def shell_sort(nums: list[int]):
    G_asc = []
    h = 1
    while h < len(nums):
        G_asc.append(h)
        h = 3 * h + 1
    G = G_asc[::-1]
    logger.debug(G)

    for g in G:
        # 呼び出し先で再代入を行わない限りは、メソッドを跨いでも同じオブジェクトが参照されるため、ここで再代入を行わない書き方も可能である。
        # しかし、呼び出し元がMutableを期待している時に、呼び出し先がImmutableな挙動だと、動かすまで誤りが分からない。
        # したがって、呼び出し元でImmutableを強制し、バグを未然に防ぐ。
        nums = insertion_sort(nums, g)

    return (len(G), G, nums)

In [11]:
def parse(input: str):
    lines = input.splitlines()
    return [int(num) for num in lines[1:]]

In [12]:
input = """5
5
1
4
3
2"""
expected = [5, 1, 4, 3, 2]
actual = parse(input)
assert expected == actual

In [15]:
shell_sort(parse(input))

[4, 1]
g=4, rem=0
g=4, rem=0, last=4
g=4, rem=0, last=4, challenged=0
nums=[2, 1, 4, 3, 5]
nums=[2, 1, 4, 3, 5]
nums=[2, 1, 4, 3, 5]
g=4, rem=1
nums=[2, 1, 4, 3, 5]
g=4, rem=2
nums=[2, 1, 4, 3, 5]
g=4, rem=3
nums=[2, 1, 4, 3, 5]
g=1, rem=0
g=1, rem=0, last=1
g=1, rem=0, last=1, challenged=0
nums=[1, 2, 4, 3, 5]
nums=[1, 2, 4, 3, 5]
g=1, rem=0, last=2
g=1, rem=0, last=2, challenged=1
nums=[1, 2, 4, 3, 5]
g=1, rem=0, last=2, challenged=0
nums=[1, 2, 4, 3, 5]
nums=[1, 2, 4, 3, 5]
g=1, rem=0, last=3
g=1, rem=0, last=3, challenged=2
nums=[1, 2, 3, 4, 5]
g=1, rem=0, last=3, challenged=1
nums=[1, 2, 3, 4, 5]
g=1, rem=0, last=3, challenged=0
nums=[1, 2, 3, 4, 5]
nums=[1, 2, 3, 4, 5]
g=1, rem=0, last=4
g=1, rem=0, last=4, challenged=3
nums=[1, 2, 3, 4, 5]
g=1, rem=0, last=4, challenged=2
nums=[1, 2, 3, 4, 5]
g=1, rem=0, last=4, challenged=1
nums=[1, 2, 3, 4, 5]
g=1, rem=0, last=4, challenged=0
nums=[1, 2, 3, 4, 5]
nums=[1, 2, 3, 4, 5]
nums=[1, 2, 3, 4, 5]


(2, [4, 1], [1, 2, 3, 4, 5])